<a href="https://colab.research.google.com/github/Pedro184294/-Pensamento-Computacional-e-Automa-o-com-Python---GS---2o-semestre/blob/main/Pensamento_Computacional_e_Automa%C3%A7%C3%A3o_com_Python_GS_2o_semestre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import json
import os
from typing import Dict, List, Tuple


class Competencia:
    def __init__(self, nome: str, peso: float = 1.0):
        self.nome = nome.lower()
        self.peso = float(peso)


class Perfil:
    def __init__(self, nome: str):
        self.nome = nome
        self.competencias: Dict[str, float] = {}

    def set_competencia(self, nome: str, score: float):
        nome = nome.lower()
        self.competencias[nome] = min(max(float(score), 0.0), 10.0)

    def get_competencia(self, nome: str) -> float:
        return float(self.competencias.get(nome.lower(), 0.0))

    def to_dict(self) -> Dict:
        return {"nome": self.nome, "competencias": self.competencias}

    @staticmethod
    def from_dict(d: Dict):
        p = Perfil(d.get("nome", "sem-nome"))
        for k, v in d.get("competencias", {}).items():
            p.set_competencia(k, float(v))
        return p



class Carreira:
    def __init__(self, nome: str, requisitos: Dict[str, Tuple[float, float]], descricao: str = ""):
        self.nome = nome
        self.requisitos = {k.lower(): (float(w), float(s)) for k, (w, s) in requisitos.items()}
        self.descricao = descricao



class Recomendador:
    def __init__(self, carreiras: List[Carreira]):
        self.carreiras = carreiras

    def fit_score(self, perfil: Perfil, carreira: Carreira) -> float:
        total_peso = soma = 0.0
        for comp, (peso, ideal) in carreira.requisitos.items():
            perfil_score = perfil.get_competencia(comp)
            dist = abs(ideal - perfil_score) / 10.0
            soma += peso * (1.0 - dist)
            total_peso += peso
        return round((soma / total_peso) * 100, 2) if total_peso > 0 else 0.0

    def recomendar(self, perfil: Perfil, top_n: int = 5) -> List[Tuple[Carreira, float, List[str]]]:
        resultados = []
        for carreira in self.carreiras:
            score = self.fit_score(perfil, carreira)
            dicas = []
            for comp, (peso, ideal) in carreira.requisitos.items():
                s = perfil.get_competencia(comp)
                if s + 1.5 < ideal:
                    dicas.append(f"Melhore '{comp}' em cerca de {ideal - s:.1f} pontos (ideal: {ideal})")
            resultados.append((carreira, score, dicas))
        resultados.sort(key=lambda x: x[1], reverse=True)
        return resultados[:top_n]



DEFAULT_CAREERS = [
    Carreira(
        "Engenheiro de Machine Learning",
        {"logica": (0.25, 9), "matematica": (0.25, 9),
         "programacao": (0.25, 8), "comunicacao": (0.1, 6), "colaboracao": (0.15, 6)},
        "Desenvolve modelos de aprendizado de máquina e analisa dados."
    ),
    Carreira(
        "Designer de Experiência (UX)",
        {"criatividade": (0.35, 9), "empatia": (0.25, 8),
         "comunicacao": (0.2, 8), "prototipacao": (0.2, 7)},
        "Cria experiências centradas no usuário e interfaces intuitivas."
    ),
    Carreira(
        "Especialista em Cibersegurança",
        {"seguranca": (0.35, 9), "redes": (0.25, 8),
         "logica": (0.15, 7), "analise": (0.25, 8)},
        "Protege sistemas e realiza auditorias de segurança digital."
    ),
    Carreira(
        "Analista de Dados",
        {"matematica": (0.2, 8), "estatistica": (0.25, 8),
         "programacao": (0.25, 7), "visualizacao": (0.15, 7),
         "comunicacao": (0.15, 7)},
        "Analisa dados e comunica insights para apoio à decisão."
    ),
    Carreira(
        "Especialista em Automação e Robótica",
        {"programacao": (0.3, 8), "eletronica": (0.25, 7),
         "logica": (0.2, 8), "projeto": (0.25, 7)},
        "Desenvolve sistemas automatizados e robôs inteligentes."
    ),
]


def criar_perfil_interativo():
    nome = input("Digite seu nome: ")
    perfil = Perfil(nome)
    print("\nInforme suas competências (nota de 0 a 10). Deixe em branco para pular.\n")

    competencias = sorted({k for c in DEFAULT_CAREERS for k in c.requisitos.keys()})
    for comp in competencias:
        val = input(f"{comp}: ")
        if not val.strip():
            continue
        try:
            perfil.set_competencia(comp, float(val))
        except ValueError:
            print("Valor inválido, ignorando...")

    return perfil


def salvar_perfil(perfil: Perfil, pasta="/content/profiles"):
    os.makedirs(pasta, exist_ok=True)
    path = os.path.join(pasta, f"{perfil.nome.replace(' ', '_')}.json")
    with open(path, "w", encoding="utf-8") as f:
        json.dump(perfil.to_dict(), f, ensure_ascii=False, indent=2)
    print(f"✅ Perfil salvo em {path}")


def gerar_recomendacoes(perfil: Perfil):
    rec = Recomendador(DEFAULT_CAREERS)
    resultados = rec.recomendar(perfil)
    print(f"\n=== Recomendações para {perfil.nome} ===\n")
    for i, (carreira, score, dicas) in enumerate(resultados, 1):
        print(f"{i}. {carreira.nome} — Aderência: {score}%")
        if dicas:
            for d in dicas:
                print("   •", d)
    print("\n✅ Análise concluída!\n")


def iniciar_sistema():
    print("=== Sistema de Orientação de Carreiras ===")
    perfil = criar_perfil_interativo()
    salvar_perfil(perfil)
    gerar_recomendacoes(perfil)




In [4]:
iniciar_sistema()

=== Sistema de Orientação de Carreiras ===
Digite seu nome: Pedro 

Informe suas competências (nota de 0 a 10). Deixe em branco para pular.

analise: 5
colaboracao: 9
comunicacao: 10
criatividade: 5
eletronica: 9
empatia: 8
estatistica: 5
logica: 10
matematica: 9
programacao: 3
projeto: 7
prototipacao: 9
redes: 6
seguranca: 4
visualizacao: 8
✅ Perfil salvo em /content/profiles/Pedro_.json

=== Recomendações para Pedro  ===

1. Designer de Experiência (UX) — Aderência: 78.0%
   • Melhore 'criatividade' em cerca de 4.0 pontos (ideal: 9.0)
2. Engenheiro de Machine Learning — Aderência: 76.5%
   • Melhore 'programacao' em cerca de 5.0 pontos (ideal: 8.0)
3. Especialista em Automação e Robótica — Aderência: 76.0%
   • Melhore 'programacao' em cerca de 5.0 pontos (ideal: 8.0)
4. Analista de Dados — Aderência: 74.5%
   • Melhore 'estatistica' em cerca de 3.0 pontos (ideal: 8.0)
   • Melhore 'programacao' em cerca de 4.0 pontos (ideal: 7.0)
5. Especialista em Cibersegurança — Aderência: 65.5%
